<a href="https://colab.research.google.com/github/reshmahambire/AI-Tutor-Generating-exercises-and-solutions-using-LLM-and-RAG/blob/main/Copy_of_AI_tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai
!pip install langchain-community
!pip install pymupdf

#Add a PDF Document Retrieval System

In [ ]:
import fitz #PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

#Upload a PDF
pdf_text = extract_text_from_pdf("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
print(pdf_text[:1000])  # Print the first 1000 characters of extracted text


Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring signiﬁcantly
less time to train. Our model a

#Preprocessing

In [ ]:
text = pdf_text.replace("\n"," ")

In [ ]:
print(text[:500])

Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.com Noam Shazeer∗ Google Brain noam@google.com Niki Parmar∗ Google Research nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.com Aidan N. Gomez∗† University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or


#Importing Necessary Libraries

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline

#Create Embedding

In [ ]:
# Load embedding model
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Extracted PDF text
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_text(text)

# Create chunk embeddings once
chunk_embeddings = np.array(embeddings.embed_documents(chunks))

#Cosine Similarity

In [ ]:
# Function to perform cosine similarity search
def search_document(query, top_k=1, threshold=0.5):
    # Embed the query
    query_embedding = np.array(embeddings.embed_query(query)).reshape(1, -1)

    # Compute cosine similarity with all chunks
    similarities = cosine_similarity(query_embedding, chunk_embeddings)[0]

    # Get top-k results above the threshold
    top_indices = np.argsort(similarities)[::-1][:top_k]

    results = [chunks[i] for i in top_indices if similarities[i] > threshold]

    return results

#Summarizer Model

In [ ]:
# Initialize summarizer here
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


#Query

In [ ]:
# Test the search
query = "What is multi-head attention?"
results = search_document(query)

#Generating Output

In [ ]:
# Summarize all results
for text in results:
    summary = summarizer(text, do_sample=False)

    print(f"\nSummary:\n{summary[0]['summary_text']}\n")

Your max_length is set to 142, but your input_length is only 107. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)



Summary:
In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions. This makes it more difﬁcult to learn dependencies between distant positions. We counteract this effect with Multi-Head Attention as described in section 3.2.

